In [1]:
from airsenal.framework.utils import *
from airsenal.framework.FPL_scoring_rules import points_for_yellow_card, points_for_red_card

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
min_minutes = 1
max_minutes = 90
min_matches = 10

query = (
    session.query(PlayerScore)
    .filter(PlayerScore.minutes >= min_minutes)
    .filter(PlayerScore.minutes <= max_minutes)
)
# TODO filter on gw and season
df = pd.read_sql(query.statement, session.bind)


In [3]:
df["yellow_cards"].value_counts(normalize=True)

0    0.885188
1    0.114812
Name: yellow_cards, dtype: float64

In [4]:
df["red_cards"].value_counts(normalize=True)

0    0.995732
1    0.004268
Name: red_cards, dtype: float64

In [5]:
df["card_pts"] = -df["yellow_cards"] - 3 * df["red_cards"]

In [6]:
card_pts = df.groupby("player_id").card_pts.sum()

match_counts = df.groupby("player_id").card_pts.count()
match_counts[match_counts < min_matches] = min_matches

sum_cards = df.groupby("player_id").card_pts.sum()

avg_cards = sum_cards / match_counts

In [7]:
players = pd.read_sql(session.query(Player).statement, session.bind)
players.set_index("player_id", inplace=True)

avg_cards = pd.merge(avg_cards, players, how="left", left_index=True, right_index=True)
avg_cards.dropna().sort_values(by="card_pts", ascending=True).head(20)

,card_pts,fpl_api_id,name
player_id,,,
398,-0.529412,294.0,Marcos Rojo
230,-0.466667,172.0,Kevin McDonald
554,-0.437500,406.0,Juan Foyth
591,-0.421053,544.0,Conor Gallagher
2,-0.409091,2.0,Sokratis Papastathopoulos
108,-0.385965,82.0,Phil Bardsley
87,-0.350000,77.0,Tariq Lamptey
444,-0.338710,334.0,Fabian Schär
232,-0.333333,174.0,Stefan Johansen
